In [1]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.datasets import load_iris, load_breast_cancer, load_wine
from sporboost.forest import *
import pandas as pd
from time import time
import os
from sklearn.metrics import make_scorer
from xgboost import XGBClassifier

################################################################################
################################################################################
################################################################################
# Data Load
def load_simulated(path):
    train = np.loadtxt(f"{path}_train.csv", delimiter=",")
    # test = np.loadtxt(f"{path}_test.csv", delimiter=",")
    data = train
    # data = np.concatenate((train, test), axis=0)
    return data[:,:-1], data[:,-1].astype('int32')

dfs = {
    'iris' : load_iris(return_X_y = True),
    'breast_cancer' : load_breast_cancer(return_X_y = True),
    'wine' : load_wine(return_X_y = True),
    'trunk' : load_simulated("source_data/simulated/Trunk"),
    'orthant' : load_simulated("source_data/simulated/Orthant"),
    'sparse_parity' : load_simulated("source_data/simulated/Sparse_parity")
}


C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
################################################################################
################################################################################
################################################################################
# Hyperparameters across all estimators
# Boosted Forest
boosted_max_depth = (2, 3)

# Sparse Random Trees
d_ratio = (0.5, 0.75, 1)
s_ratio = (2., 3., 4.)

# Rotational Decision Trees
K = (2, 3)


In [3]:
################################################################################
################################################################################
################################################################################
# Scoring Functions
scoring = {
    "cohens_kappa" : make_scorer(cohen_kappa_score),
    "accuracy" : 'accuracy'
}

In [4]:
################################################################################
################################################################################
################################################################################
# Initalize cross-validation params
pipe = Pipeline([
    ('clf', RandomForest())
])

params = [
    {
        'clf' : (RandomForest(),),
    },
    {
        'clf' : (AdaBoost(),),
        'clf__max_depth' : boosted_max_depth
    },
    {
        'clf' : (SPORF(),),
        'clf__d_ratio' : d_ratio,
        'clf__s_ratio' : s_ratio
    },
    {
        'clf' : (SPORBoost(),),
        'clf__max_depth' : boosted_max_depth,
        'clf__d_ratio' : d_ratio,
        'clf__s_ratio' : s_ratio
    },
    {
        'clf' : (RotationalForest(),),
        'clf__K' : K
    },
    {
        'clf' : (RotBoost(),),
        'clf__max_depth' : boosted_max_depth,
        'clf__K' : K
    },
    {
        'clf' : (SPORBoostHC(),),
        'clf__max_depth' : boosted_max_depth,
        'clf__d_ratio' : d_ratio,
        'clf__s_ratio' : s_ratio
    },
    {
        'clf' : (XGBClassifier(nthread = 1, use_label_encoder=False, max_depth=3), )
    }
]

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=20220329)

cv_base = GridSearchCV(estimator = pipe,
                       param_grid = params,
                       cv=kfold,
                       n_jobs=-1,
                       verbose=3,
                       refit=False,
                       scoring=scoring
)


In [5]:
################################################################################
################################################################################
################################################################################
# Func to write results to disk
def write_results(cv, label):
    front_cols = ['param_clf',
                  'mean_test_cohens_kappa', 'std_test_cohens_kappa',
                  'mean_test_accuracy', 'std_test_accuracy',
                  'param_clf__max_depth',
                  'param_clf__d_ratio', 'param_clf__s_ratio',
                  'param_clf__K'
                ]

    results = pd.DataFrame(cv.cv_results_).drop(['params'], axis=1)
    results['param_clf'] = results['param_clf'].apply(lambda c: type(c).__name__)
    results = results.sort_values(['param_clf', 'mean_test_cohens_kappa'],
                                ascending=False, ignore_index = True)
            
    results = results[front_cols + results.drop(front_cols, axis=1).columns.tolist()]

    # All Results
    results.to_csv(f"report/{label}_all.csv", index=False)

    # Best Results
    results.groupby("param_clf").nth(0).reset_index(drop=False).to_csv(f"report/{label}_best.csv", index=False)

In [6]:
################################################################################
################################################################################
################################################################################
# Fit the CVs
for name, df in dfs.items():
    if not os.path.exists(f"report/{name}_all.csv"):
        start = time()
        print(f"Testing {name}")
        res = cv_base.fit(*df)
        write_results(cv_base.fit(*df), name)
        end = time()
        print(f"Finished {name} in {(end - start) / 60:.1f} mins")
    else:
        print(f"Skipping {name} as it has already been tested")

Testing iris
Fitting 10 folds for each of 55 candidates, totalling 550 fits
Fitting 10 folds for each of 55 candidates, totalling 550 fits


C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.94 0.94 0.92 0.95 0.95 0.94 0.95 0.95 0.95 0.94 0.95 0.94 0.94 0.96
 0.95 0.95 0.95 0.94 0.93 0.94 0.95 0.94 0.95 0.95 0.94 0.95 0.95 0.94
 0.94 0.95 0.95 0.94 0.95 0.94 0.95 0.94 0.93 0.94 0.97 0.95 0.95 0.95
 0.95 0.95 0.96 0.95 0.94 0.94 0.93 0.94 0.95  nan 0.95 0.94 0.93]
  warnings.warn(
C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.96       0.96       0.94666667 0.96666667 0.96666667 0.96
 0.96666667 0.96666667 0.96666667 0.96       0.96666667 0.96
 0.96       0.97333333 0.96666667 0.96666667 0.96666667 0.96
 0.95333333 0.96       0.96666667 0.96       0.96666667 0.96666667
 0.96       0.96666667 0.96666667 0.96       0.96       0.96666667
 0.96666667 0.96       0.96666667 0.96       0.96666667 0.96
 0.95333333 0.96       0.

Finished iris in 2.3 mins
Testing breast_cancer
Fitting 10 folds for each of 55 candidates, totalling 550 fits
Fitting 10 folds for each of 55 candidates, totalling 550 fits
Finished breast_cancer in 27.8 mins
Testing wine
Fitting 10 folds for each of 55 candidates, totalling 550 fits
Fitting 10 folds for each of 55 candidates, totalling 550 fits
Finished wine in 3.6 mins
Testing trunk
Fitting 10 folds for each of 55 candidates, totalling 550 fits


C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.78 0.8  0.74 0.96 0.92 1.   0.92 0.92 0.98 0.92 0.94 0.96 0.96 1.
 0.98 0.94 0.96 0.94 0.98 0.98 0.98 0.98 0.98 0.96 0.96 0.98 0.98 0.98
 0.98 0.96 0.96 0.98  nan  nan  nan  nan 0.96 0.98 0.96 0.96 0.98 0.98
 0.94 0.96 0.94 0.96 0.96 0.98 0.96 0.96 1.   0.96 0.96 0.96 0.86]
  warnings.warn(
C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.89 0.9  0.87 0.98 0.96 1.   0.96 0.96 0.99 0.96 0.97 0.98 0.98 1.
 0.99 0.97 0.98 0.97 0.99 0.99 0.99 0.99 0.99 0.98 0.98 0.99 0.99 0.99
 0.99 0.98 0.98 0.99  nan  nan  nan  nan 0.98 0.99 0.98 0.98 0.99 0.99
 0.97 0.98 0.97 0.98 0.98 0.99 0.98 0.98 1.   0.98 0.98 0.98 0.93]
  warnings.warn(


Fitting 10 folds for each of 55 candidates, totalling 550 fits


C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.8  0.8  0.74 0.98 0.94 0.98 0.94 0.94 0.96 0.9  0.92 0.94 0.98 0.98
 0.96 0.98 0.98 0.96 0.94 0.98 0.98 0.98 1.   0.96 0.96 0.96 0.98 0.94
 0.96 0.96 0.98 0.98  nan  nan  nan  nan 1.   0.98 0.96 0.96 0.98 1.
 0.96 0.98 1.   0.98 0.98 0.98 0.94 0.94 0.98 0.96 0.94 0.98 0.86]
  warnings.warn(
C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.9  0.9  0.87 0.99 0.97 0.99 0.97 0.97 0.98 0.95 0.96 0.97 0.99 0.99
 0.98 0.99 0.99 0.98 0.97 0.99 0.99 0.99 1.   0.98 0.98 0.98 0.99 0.97
 0.98 0.98 0.99 0.99  nan  nan  nan  nan 1.   0.99 0.98 0.98 0.99 1.
 0.98 0.99 1.   0.99 0.99 0.99 0.97 0.97 0.99 0.98 0.97 0.99 0.93]
  warnings.warn(


Finished trunk in 17.4 mins
Testing orthant
Fitting 10 folds for each of 55 candidates, totalling 550 fits
Fitting 10 folds for each of 55 candidates, totalling 550 fits
Finished orthant in 73.8 mins
Testing sparse_parity
Fitting 10 folds for each of 55 candidates, totalling 550 fits
Fitting 10 folds for each of 55 candidates, totalling 550 fits
Finished sparse_parity in 16.4 mins
